# Boosting with California Housing Dataset

## California Housing Dataset

### Downloading the Dataset Using Scikit-Learn Directly:

In [ ]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()

### Using the Provided Pickle File:

In [ ]:
# from google.colab import drive
# drive.mount("/content/gdrive")

In [ ]:
# import os
# import pickle

# # root_dir = "PATH/TO/YOUR/DIRECTORY"

# # Checking if our specified directory exists
# print(os.path.exists(root_dir))

# with open(os.path.join(root_dir, 'california_housing.pkl'), 'rb') as f:
#     housing = pickle.load(f)

In [ ]:
housing.keys()

In [ ]:
print(housing.DESCR)

In [ ]:
housing.feature_names

In [ ]:
housing.data[:5]

In [ ]:
housing.target_names

In [ ]:
housing.target

## Data Preprocessing

In [ ]:
import pandas as pd

target = housing.target_names[0]

df = pd.DataFrame(housing.data, columns=housing.feature_names)
df[target] = housing.target
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
X = pd.DataFrame(housing.data, columns=housing.feature_names)
y = pd.Series(housing.target)

In [ ]:
import matplotlib.pyplot as plt

# Create individual scatter plots for each input variable vs. the output variable
plt.figure(figsize=(12, 6))

for i, feature in enumerate(X.columns):
    plt.subplot(2, 4, i + 1)  # Adjust the subplot layout as needed
    plt.scatter(X[feature], y, alpha=0.3)
    plt.title("{} vs. {}".format(feature, target))
    plt.xlabel(feature)
    plt.ylabel(target)

plt.tight_layout()
plt.show()

### Outlier Removal

* Winzorize ([scipy.stats.mstats.winsorize](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.mstats.winsorize.html))

In [ ]:
from scipy.stats.mstats import winsorize

feature_with_outliers = ["AveOccup"]

# Define the percentile thresholds for Winsorization
lower_percentile = 0.0
upper_percentile = 0.0002

# Apply Winsorization to each input variable
for feature in X.columns:
    if feature in feature_with_outliers:
      X[feature] = winsorize(X[feature], limits=(lower_percentile, upper_percentile))

In [ ]:
# Create individual scatter plots for each input variable vs. the output variable
plt.figure(figsize=(12, 6))

for i, feature in enumerate(X.columns):
    plt.subplot(2, 4, i + 1)  # Adjust the subplot layout as needed
    plt.scatter(X[feature], y, alpha=0.3)
    plt.title("{} vs. {}".format(feature, target))
    plt.xlabel(feature)
    plt.ylabel(target)

plt.tight_layout()
plt.show()

* 2D hexagonal binning plot ([matplotlib.pyplot.hexbin](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.hexbin.html))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

# Define a z-score threshold for outlier detection (e.g., 3 standard deviations)
z_score_threshold = 3

# Calculate the z-scores for each feature
z_scores = np.abs(stats.zscore(X))

# Identify rows where all features are within the threshold
row_mask = (z_scores < z_score_threshold).all(axis=1)

# Filter X and y based on the row mask
X_no_outliers = X[row_mask]
y_no_outliers = y[row_mask]

# Create hexbin plots for each input variable vs. the output variable
plt.figure(figsize=(12, 6))

for i, feature in enumerate(X_no_outliers.columns):
    plt.subplot(2, 4, i + 1)
    _X, _y = X_no_outliers[feature], y_no_outliers
    plt.hexbin(_X, _y, gridsize=30, cmap="Blues")
    plt.title("{} vs. {}".format(feature, target))
    plt.xlabel(feature)
    plt.ylabel(target)

plt.tight_layout()
plt.show()

### Data Split

In [ ]:
from sklearn.model_selection import train_test_split

random_state = 100
shuffle = True
test_size_ratio = 0.2

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size_ratio, random_state=random_state, shuffle=shuffle)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

## Training & Validation

Cross Validation
* K-folds cross validator ([model_selection.KFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html))
* Grid search over specified parameter values ([model_selection.GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html))

In [ ]:
from sklearn.model_selection import KFold, GridSearchCV

You can change the scoring function by using the `scoring` parameter for cross validation.
* `r2`: $R^2$ score
* `neg_mean_squared_error`: Negative mean squared error (MSE)
* `neg_mean_absolute_error`: Negative mean absolute error (MAE)

Note that the negative values for MSE, MAE, and RMSE are used by convention because `cross_val_score` or `GridSearchCV` is designed to prefer higher scoring metrics.

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
scoring = "neg_mean_squared_error"

models = {}

### 1. Decision Tree
* [sklearn.tree.DecisionTreeRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html)

In [ ]:
from sklearn.tree import DecisionTreeRegressor

model = DecisionTreeRegressor(random_state=random_state)

# Define the hyperparameters and their possible values
param_grid = {
    "max_depth": [5, 10, 20],
    "min_samples_split": [2, 10, 20],
    "ccp_alpha": [0.0, 0.01],
}

grid_search = GridSearchCV(model, param_grid, cv=kf, scoring=scoring, refit=True, n_jobs=-1)
grid_search.fit(X_train, y_train)

print("Best parameters: ", grid_search.best_params_)
print("Best CV score: {:.6f}".format(grid_search.best_score_))

models["Decision Tree"] = grid_search.best_estimator_

### 2. AdaBoost
* [sklearn.ensemble.AdaBoostRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostRegressor.html)

In [ ]:
from sklearn.ensemble import AdaBoostRegressor

model = AdaBoostRegressor(n_estimators=50,
                          loss="linear",
                          random_state=random_state)

# Define the hyperparameters and their possible values
param_grid = {
    "estimator": [DecisionTreeRegressor(max_depth=3), DecisionTreeRegressor(max_depth=6)],
    "learning_rate": [0.1, 1.0],
}

grid_search = GridSearchCV(model, param_grid, cv=kf, scoring=scoring, refit=True, n_jobs=-1)
grid_search.fit(X_train, y_train)

print("Best parameters: ", grid_search.best_params_)
print("Best CV score: {:.6f}".format(grid_search.best_score_))

models["AdaBoost"] = grid_search.best_estimator_

### 3. Gradient Boosting
* [sklearn.ensemble.GradientBoostingRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

model = GradientBoostingRegressor(n_estimators=50,
                                  loss="squared_error",
                                  subsample=1.0,
                                  random_state=random_state)

# Define the hyperparameters and their possible values
param_grid = {
    "max_depth": [3, 6],
    "learning_rate": [0.0, 0.1],
}

grid_search = GridSearchCV(model, param_grid, cv=kf, scoring=scoring, refit=True, n_jobs=-1)
grid_search.fit(X_train, y_train)

print("Best parameters: ", grid_search.best_params_)
print("Best CV score: {:.6f}".format(grid_search.best_score_))

models["Gradient Boosting"] = grid_search.best_estimator_

### 4. XGBoost
* [xgboost.XGBRegressor](https://xgboost.readthedocs.io/en/stable/python/python_api.html#xgboost.XGBRegressor)

In [ ]:
from xgboost import XGBRegressor

model = XGBRegressor(n_estimators=50,
                     subsample=1.0,
                     learning_rate=0.1,
                     max_depth=6,
                     n_jobs=-1,
                     random_state=random_state)

# Define the hyperparameters and their possible values
param_grid = {
    "reg_alpha": [0, 0.1],
    "reg_lambda": [0, 0.1],
}

grid_search = GridSearchCV(model, param_grid, cv=kf, scoring=scoring, refit=True)
grid_search.fit(X_train, y_train)

print("Best parameters: ", grid_search.best_params_)
print("Best CV score: {:.6f}".format(grid_search.best_score_))

models["XGBoost"] = grid_search.best_estimator_

### 5. LightGBM
* [lightgbm.LGBMRegressor](https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMRegressor.html)

In [ ]:
from lightgbm import LGBMRegressor

model = LGBMRegressor(n_estimators=50,
                      learning_rate=0.1,
                      data_sample_strategy="goss",
                      top_rate=0.2,
                      other_rate=0.1,
                      force_col_wise=True,
                      verbosity=0,
                      n_jobs=-1,
                      random_state=random_state)

# Define the hyperparameters and their possible values
param_grid = {
    "reg_alpha": [0, 0.1],
    "reg_lambda": [0, 0.1],
    "enable_bundle": [True, False]
}

grid_search = GridSearchCV(model, param_grid, cv=kf, scoring=scoring, refit=True)
grid_search.fit(X_train, y_train)

print("Best parameters: ", grid_search.best_params_)
print("Best CV score: {:.6f}".format(grid_search.best_score_))

models["LightGBM"] = grid_search.best_estimator_

### 6. Stacking
* [sklearn.ensemble.StackingRegressor](https://scikit-learn.org/dev/modules/generated/sklearn.ensemble.StackingRegressor.html#sklearn.ensemble.StackingRegressor)

In [ ]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Ridge

base_models = list(models.items())

# Define the meta-learner for stacking
meta_learner = Ridge()

# Create the stacking regressor
stacking_regressor = StackingRegressor(estimators=base_models, final_estimator=meta_learner, cv="prefit", passthrough=False)
stacking_regressor.fit(X_train, y_train)

# Add the stacking model to the models dictionary
models["Stacking"] = stacking_regressor

## Evaluation

* MSE ([sklearn.metrics.mean_squared_error](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html))
* MAE ([sklearn.metrics.mean_absolute_error](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_absolute_error.html))
* $R^2$ ([sklearn.metrics.r2_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html))

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

for _name, _model in models.items():
    y_pred = _model.predict(X_test)
    print("{:>17}: MSE={:.4f} | MAE={:.4f} | R^2={:.4f}".format(
        _name,
        mean_squared_error(y_test, y_pred),
        mean_absolute_error(y_test, y_pred),
        r2_score(y_test, y_pred)
        )
    )

## Interpretation

### 1. Decision Tree
* Plot a decision tree ([tree.plot_tree](https://scikit-learn.org/stable/modules/generated/sklearn.tree.plot_tree.html))

In [ ]:
from sklearn.tree import plot_tree

_tree = DecisionTreeRegressor(max_depth=3)
_tree.fit(X_train, y_train)

plt.figure(figsize=(20, 10))
plot_tree(_tree, filled=True, feature_names=X.columns, rounded=True)
plt.show()

### 2. AdaBoost

In [ ]:
model_name = "AdaBoost"

feature_importances = models[model_name].feature_importances_
sorted_idx = np.argsort(feature_importances)[::-1]

plt.figure(figsize=(8, 4))
plt.title(model_name)
plt.bar(range(len(feature_importances)), feature_importances[sorted_idx], align="center")
plt.xticks(range(len(feature_importances)), X.columns[sorted_idx], rotation=45)
plt.xlabel("Feature")
plt.ylabel("Importance")
plt.tight_layout()
plt.show()

### 3. Gradient Boosting

In [ ]:
model_name = "Gradient Boosting"

feature_importances = models[model_name].feature_importances_
sorted_idx = np.argsort(feature_importances)[::-1]

plt.figure(figsize=(8, 4))
plt.title(model_name)
plt.bar(range(len(feature_importances)), feature_importances[sorted_idx], align="center")
plt.xticks(range(len(feature_importances)), X.columns[sorted_idx], rotation=45)
plt.xlabel("Feature")
plt.ylabel("Importance")
plt.tight_layout()
plt.show()

### 4. XGBoost

* Feature importance ([xgboost.plot_importance](https://xgboost.readthedocs.io/en/stable/python/python_api.html#xgboost.plot_importance))

In [ ]:
import xgboost as xgb

model_name = "XGBoost"
xgb.plot_importance(models[model_name],
                    importance_type="gain", # ["weight", "gain", "cover"]
                    title=model_name)
plt.show()

### 5. LightGBM

* Feature importance ([lightgbm.plot_importance](https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.plot_importance.html))

In [ ]:
import lightgbm as lgb

model_name = "LightGBM"
lgb.plot_importance(models[model_name],
                    importance_type="gain", # ["split", "gain"]
                    title=model_name)
plt.show()

### 6. Stacking

In [ ]:
trained_meta_learner = models["Stacking"].final_estimator_

# Check if the meta-learner has coefficients (like Ridge or Linear Regression)
if hasattr(trained_meta_learner, "coef_"):
    weights = trained_meta_learner.coef_
    base_model_names = [name for name, _ in models["Stacking"].estimators]

    # Print the weights along with the corresponding base model names
    for model_name, weight in zip(base_model_names, weights):
        print(f"{model_name}: {weight:.4f}")
else:
    print("The meta-learner does not have coefficients to display.")